In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import glob
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.lidar
import strawb.tools

import plotly
import plotly.graph_objs as go
import plotly.express as px
import pandas
import plotly.io as pio
pio.templates.default = "plotly_white"

import random

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

# Get a working file
## Option 1:
Get file from DB and df5 attribute measurement_type

In [ ]:
# in case execute db.load_entire_db_from_ONC() to load the entire db
db = strawb.SyncDBHandler(file_name='Default')  # loads the db

In [ ]:
# mask by device
mask = db.dataframe['deviceCode'] == 'TUMLIDAR001'
mask |= db.dataframe['deviceCode'] == 'TUMLIDAR002'

# and add mask by 'measurement_type'
mask &= db.dataframe['measurement_type'] == 'laser_adjustment'

db.dataframe[mask]  # plot the table

# Option 2
search in the strawb.Config.raw_data_dir for all files with the a given pattern

In [ ]:
def find_files():
    # Get all files available
    # takes one file randomly, takes some seconds
    file_list = strawb.sensors.lidar.FileHandler.find_files('*-SDAQ-LIDAR.hdf5',
                                                            directory=strawb.Config.raw_data_dir,
                                                            recursive=True,
                                                            raise_nothing_found=True)
    return file_list

## Helper function

In [ ]:
def get_working_file_randomly(file_list, file_version=1):
    """Some files fails with the file_handler, i.e., if a group is missing for older SDAQ versions.
    To get one file randomly which works, try it until it works.
    """

    success = False
    file_list_trials = file_list.copy()
    while len(file_list_trials) > 0 and not success:
        i = np.random.randint(0,len(file_list_trials))
        full_path = file_list_trials[i]
        try:
            lidar = strawb.sensors.lidar.FileHandler(full_path)
        except Exception as a:
            file_list_trials.pop(i)
            print(f'Error: {full_path} failed with: {a}')
        else:
            print(f'Success: {full_path}, file_version={lidar.file_version}')
            if lidar.file_version >= file_version:
                return full_path
            
    return None
        

## Get a working file

In [ ]:
file_version = 3  # select one of [1, 2, 3]

if True:
    file_list = db.dataframe[mask]['fullPath'].to_list()
else:
    file_list = find_files()

full_path = get_working_file_randomly(file_list, file_version)  # select a random file

print(f'Use: {full_path}')

## Load the file

In [ ]:
lidar = strawb.sensors.lidar.Lidar(file=full_path)
lidar.file_handler.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

# Set up plotting function

In [ ]:
def multiline_plot(data_frame):
    columns = list(set(data_frame.columns).difference(['time']))
    
    fig = plotly.subplots.make_subplots(shared_xaxes=True,
                                        rows=len(columns),
                                        cols=1, 
                                        #subplot_titles=columns
                                       )


    fig.update_layout(
        width=800,
        height=150+100*len(columns),)

    j = 1
    for i, column_i in enumerate(columns):
        fig.add_trace(go.Scatter(x=data_frame['time'], 
                                 y=data_frame[column_i],
                                 name=column_i), 
                      row=i+1, col=1
                     )

    return fig

# Plotting with pandas and plotly

In [ ]:
df_daq = lidar.file_handler.get_pandas_daq()
df_gimbal = lidar.file_handler.get_pandas_gimbal()
df_laser = lidar.file_handler.get_pandas_laser()

df_tot = lidar.file_handler.get_pandas_tot()
df_counts = lidar.file_handler.get_pandas_counts()

df_measurement = lidar.file_handler.get_pandas_measurement()

df_dcounts = lidar.pds.get_pandas_dcounts()
df_rates = lidar.pds.get_pandas_rate()

# Basic plots direct from file

In [ ]:
fig = multiline_plot(df_daq)
fig.show()

In [ ]:
fig = multiline_plot(df_gimbal)
fig.show()

In [ ]:
fig = multiline_plot(df_laser)
fig.show()

In [ ]:
fig = multiline_plot(df_measurement)
fig.show()

In [ ]:
# # Don't plot TOT data, it's a lot of data
# fig = multiline_plot(df_tot)
# fig.show()

df_tot.shape

In [ ]:
# # Don't plot counter data, it's less but 100Hz has some size.
# # Moreover those values have to be processed further. -> df_rates
# fig = multiline_plot(df_counter)
# fig.show()

df_counts.shape

# Counts and Rates

In [ ]:
# dcounts are the diff of the stored TRB counts readings. As the readings can jitter a bit, dcounts_pmt jitter, too
# to correct devide by dcounts_time to get a rate (or take df_rates) or 
# by dcounts_laser to get a success rate [events/trigger]
fig = multiline_plot(df_dcounts)
fig.show()

In [ ]:
fig = multiline_plot(df_rates)
fig.show()